In [20]:
import pandas as pd
import os
import re

In [40]:
file_paths = [
    r"E:\Sales closing Dump\Sept'20.xlsx",
    r"E:\Sales closing Dump\Apr'2020.xlsx",
    r"E:\Sales closing Dump\Aug'2020.xlsx",
    r"E:\Sales closing Dump\Dec'20.xlsx",
    r"E:\Sales closing Dump\Feb'21.xlsx",
    r"E:\Sales closing Dump\Jan'21.xlsx",
    r"E:\Sales closing Dump\Jul'20.xlsx",
    r"E:\Sales closing Dump\Jun'20.xlsx",
    r"E:\Sales closing Dump\Mar'21.xlsx",
    r"E:\Sales closing Dump\May'20.xlsx",
    r"E:\Sales closing Dump\Nov'20.xlsx",
    r"E:\Sales closing Dump\Oct'20.xlsx"]

In [42]:
def extract_month(file_path):
    # Extract the file name without path
    file_name = os.path.basename(file_path)
    # Regex to extract the month part of the file name
    match = re.search(r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w']*\s*\d{2,4}", file_name)
    return match.group(0) if match else None

In [44]:
dataframes = []

for file in file_paths:
    # Read the Excel file
    df = pd.read_excel(file)
    # Extract the month and add it as a new column
    df['Month'] = extract_month(file)
    # Append the dataframe to the list
    dataframes.append(df)


In [45]:
final_df = pd.concat(dataframes, ignore_index=True)


In [69]:
final_df.columns

Index(['Zone', 'Region', 'Area', 'Customer', ' 'Customer Name',
       'Account Group', 'Customer Class', 'Customer Classific.',
       'Customer Classific..1', 'Inches', 'Material', 'Discription', 'Pattern',
       'Billing type', 'Total Net Qty', 'Month', 'Product_type', 'Quarter'],
      dtype='object')

In [48]:
final_df['Month'].value_counts()

Month
Oct'20      118440
Sept'20     116419
Dec'20      116331
Mar'21      115863
Jan'21      115036
Feb'21      112688
Nov'20      110983
Jul'20      104969
Aug'2020    104910
Jun'20      102410
May'20       69452
Apr'2020     11392
Name: count, dtype: int64

In [100]:
def determine_product_type(value_or_material):
    # First, handle the PCR-BIAS and PCR-RADIAL conditions (from the first function)
    if value_or_material.startswith("131") or value_or_material.startswith("133") or value_or_material.startswith("141") or value_or_material.startswith("143"):
        return "PCR-BIAS"
    elif value_or_material.startswith("132") or value_or_material.startswith("142") or value_or_material.startswith("145"):
        return "PCR-RADIAL"
    elif value_or_material.startswith("121") or value_or_material.startswith("123"):
        return "LCV-BIAS"
    elif value_or_material.startswith("122") or value_or_material.startswith("125"):
        return "LCV-RADIAL"
    elif value_or_material.startswith("1D1") or value_or_material.startswith("1D3"):
        return "SCV-BIAS"
    elif value_or_material.startswith("1D2") or value_or_material.startswith("1D5"):
        return "SCV-RADIAL"
    elif value_or_material.startswith("1S"):
        return "SCOOTER"
    elif value_or_material.startswith("1B"):
        return "MOTORCYCLE"
    elif value_or_material.startswith("1H"):
        return "3W"
    elif value_or_material.startswith("111") or value_or_material.startswith("113"):
        return "TRUCK-BIAS"
    elif value_or_material.startswith("112") or value_or_material.startswith("115"):
        return "TRUCK-RADIAL"
    elif value_or_material.startswith("1E1") or value_or_material.startswith("181"):
        return "OTR"
    elif value_or_material.startswith("15") or value_or_material.startswith("16") or value_or_material.startswith("17") or value_or_material.startswith("19"):
        return "FARM"
    else:
        return "NA"


In [102]:
final_df['Product_type'] = final_df['Material'].apply(determine_product_type)

In [128]:
final_df['Customer Classific.'].value_counts()

Customer Classific.
PD            224625
CO            190070
SW            161677
NT            125773
DB            124329
TP            112875
TR             70744
XW             50858
PP             39481
RD             14529
IC             12122
TW              5135
FM              5058
HC              4861
#               4072
FW              3468
OT              3396
CP              3090
MS              2311
WV              1444
Non truck       1329
EC              1288
1                773
PS               434
MB               322
RN               269
01               127
EN               125
2P                87
IP                45
HD                45
M4                31
NS                19
HY                15
HT                 8
M5                 6
IN                 4
B-customer         2
N5                 1
Name: count, dtype: int64

In [108]:
def assign_quarter(month):
    month_lower = month.lower()
    
    # Check for Q4: Jan'21, Feb'21, Mar'21
    if "jan'21" in month_lower or "feb'21" in month_lower or "mar'21" in month_lower:
        return "Q4"
    
    # Check for Q1: Apr'2020, May'20, Jun'20
    elif "apr'2020" in month_lower or "may'20" in month_lower or "jun'20" in month_lower:
        return "Q1"
    
    # Check for Q2: Jul'20, Aug'2020, Sept'20
    elif "jul'20" in month_lower or "aug'2020" in month_lower or "sept'20" in month_lower:
        return "Q2"
    
    # Check for Q3: Oct'20, Nov'20, Dec'20
    elif "oct'20" in month_lower or "nov'20" in month_lower or "dec'20" in month_lower:
        return "Q3"
    
    else:
        return "Unknown"


In [112]:
final_df['Quarter'] = final_df['Month'].apply(assign_quarter)


In [114]:
df = pd.read_excel(r"E:\Sales closing Dump\SAP id in score card.xlsx")

In [116]:
final_df['Exists'] = final_df['Customer'].isin(df['SAP ID']).map({True: 'Yes', False: 'No'})


In [118]:
final_df.columns

Index(['Zone', 'Region', 'Area', 'Customer', ' 'Customer Name',
       'Account Group', 'Customer Class', 'Customer Classific.',
       'Customer Classific..1', 'Inches', 'Material', 'Discription', 'Pattern',
       'Billing type', 'Total Net Qty', 'Month', 'Product_type', 'Quarter',
       'Exists'],
      dtype='object')

In [134]:
filtered_df = final_df[
    (final_df['Exists'] == 'Yes') &
    (final_df['Product_type'] == 'PCR-RADIAL') &
    ((final_df['Account Group'] == 'Z001') | 
     ((final_df['Account Group'] == 'Z004') & final_df['Customer Classific.'].isin(['DB', 'RD']))) &
    (final_df['Billing type'].isin(['ZOR', 'YOR', 'YPLT', 'ZPLT', 'ZRDR', 'YRDR']))
]

pivot_table = filtered_df.pivot_table(
    index='Quarter',
    values=['Customer', 'Total Net Qty'],
    aggfunc={'Customer': 'count', 'Total Net Qty': 'sum'},
    fill_value=0
)

print(pivot_table)


         Customer  Total Net Qty
Quarter                         
Q1          29447         175692
Q2          70258         534838
Q3          85191         682937
Q4          83610         627940


In [138]:
filtered_df = final_df[
    (final_df['Exists'] == 'Yes') &
    (final_df['Product_type'] == 'PCR-RADIAL') &
    ((final_df['Account Group'] == 'Z001') | 
     ((final_df['Account Group'] == 'Z004') & final_df['Customer Classific.'].isin(['DB', 'RD'])))
]

pivot_table = filtered_df.pivot_table(
    index='Quarter',
    values=['Customer', 'Total Net Qty'],
    aggfunc={'Customer': 'count', 'Total Net Qty': 'sum'},
    fill_value=0
)

print(pivot_table)



         Customer  Total Net Qty
Quarter                         
Q1          32749         179896
Q2          81507         551985
Q3          96539         700396
Q4          95572         645347
